In [1]:
!pip install hazm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 16.1 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=0a9bd3a62251c8cdbb2a78a431ad9ea243a359d7a7410c467e14cbdaaaa6a03d
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import random

# ===============================
# 1. تعریف توابع داده‌افزایی
# ===============================

def random_deletion(sentence, p=0.2):
    """
    حذف تصادفی کلمات در جمله با احتمال p.
    اگر جمله تنها یک کلمه داشته باشد، همان جمله برگردانده می‌شود.
    """
    words = sentence.split()
    if len(words) == 1:
        return sentence
    new_words = [word for word in words if random.random() > p]
    # در صورت خالی شدن لیست، یک کلمه تصادفی انتخاب شود.
    if not new_words:
        return random.choice(words)
    return " ".join(new_words)

def random_swap(sentence, n_swaps=2):
    """
    جابجایی تصادفی n_swaps بار دو کلمه در جمله.
    """
    words = sentence.split()
    if len(words) < 2:
        return sentence
    new_words = words.copy()
    for _ in range(n_swaps):
        idx1, idx2 = random.sample(range(len(words)), 2)
        new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
    return " ".join(new_words)

# یک دیکشنری نمونه از مترادف‌ها؛ بسته به نیاز و دیکشنری‌های موجود در زبان فارسی می‌توانید آن را گسترش دهید.
synonyms_dict = {
    "خوب": ["عالی", "بی‌نظیر", "فوق‌العاده"],
    "بد": ["نامناسب", "ضعیف", "افتضاح"],
    "زیبا": ["قشنگ", "دلنشین", "جذاب"],
    "کیفیت": ["وضوح", "دقت", "استاندارد"],
    "مشکل": ["ایراد", "نقص", "چالش"],
}

def synonym_replacement(sentence, n=1):
    """
    جایگزینی حداکثر n کلمه که در دیکشنری مترادف‌ها موجود است، با یکی از مترادف‌هایشان.
    """
    words = sentence.split()
    new_words = words.copy()
    random_words = list(set(words))  # انتخاب کلمات یکتا
    random.shuffle(random_words)
    num_replaced = 0
    for word in random_words:
        if word in synonyms_dict:
            replacement = random.choice(synonyms_dict[word])
            # جایگزینی تمام موارد کلمه مورد نظر
            new_words = [replacement if w == word else w for w in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return " ".join(new_words)

neutral_words = ["البته", "در واقع", "به طور کلی", "می‌توان گفت", "همچنین"]

def add_neutral_word(sentence, n=1):
    """
    افزودن به صورت تصادفی n کلمه خنثی به جمله.
    """
    words = sentence.split()
    for _ in range(n):
        insert_idx = random.randint(0, len(words))
        words.insert(insert_idx, random.choice(neutral_words))
    return " ".join(words)

def augment_text(sentence):
    """
    اعمال چند روش داده‌افزایی بر روی یک جمله و برگرداندن لیستی از نمونه‌های افزوده‌شده.
    """
    augmented_set = set()

    # اعمال روش‌های مختلف؛ در صورت نیاز می‌توانید ترکیب یا ترتیب اجرای آن‌ها را تغییر دهید.
    augmented_set.add(random_deletion(sentence, p=0.2))
    augmented_set.add(random_swap(sentence, n_swaps=2))
    augmented_set.add(synonym_replacement(sentence, n=1))
    augmented_set.add(add_neutral_word(sentence, n=1))

    # می‌توانید جمله اصلی را هم اضافه کنید:
    # augmented_set.add(sentence)

    return list(augmented_set)

# ===============================
# 2. اعمال داده‌افزایی بر روی دیتاست
# ===============================

# بارگذاری دیتاست اصلی (فرض می‌کنیم فایل "Finall_comments.csv" در مسیر مشخص قرار دارد)
df = pd.read_csv("/content/Finall_comments.csv")

# لیست‌هایی برای ذخیره داده‌های افزوده‌شده
augmented_texts = []
augmented_labels = []

# اعمال داده‌افزایی برای هر نمونه
for index, row in df.iterrows():
    text = row['comment']
    label = row['sentiment']
    # تولید چندین نمونه افزوده‌شده برای هر متن
    augmented_samples = augment_text(text)

    # ذخیره نمونه‌های افزوده‌شده همراه با برچسب اصلی
    for aug_text in augmented_samples:
        augmented_texts.append(aug_text)
        augmented_labels.append(label)

# ایجاد DataFrame جدید برای داده‌های افزوده‌شده
augmented_df = pd.DataFrame({
    'comment': augmented_texts,
    'sentiment': augmented_labels
})

# ترکیب دیتاست اصلی با داده‌های افزوده‌شده (در صورت تمایل)
combined_df = pd.concat([df, augmented_df], ignore_index=True)

# ذخیره دیتاست نهایی در فایل CSV
combined_df.to_csv("/content/augmented_dataset.csv", index=False)

print("داده‌افزایی انجام شد. دیتاست ترکیبی دارای {} نمونه است.".format(combined_df.shape[0]))


FileNotFoundError: [Errno 2] No such file or directory: '/content/Finall_comments.csv'

In [2]:
import re
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
def clean_text(text):
    text = text.strip()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def tokenize_text(text):
    return text.split()

import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm  # اضافه کردن tqdm برای نوار پیشرفت

# بررسی اینکه آیا GPU در دسترس است
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# بارگذاری مدل و توکنایزر روی GPU
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")
model = AutoModel.from_pretrained("BAAI/bge-m3").to(device)

def text_to_sequence(text, max_len):
    # توکنایز کردن متن
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_len)

    # انتقال ورودی‌ها به GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # استفاده از امبدینگ‌های آخرین لایه
    embeddings = outputs.last_hidden_state.mean(dim=1)  # میانگین گرفتن از توکن‌ها

    # انتقال امبدینگ‌ها به CPU و تبدیل به لیست
    seq = embeddings.squeeze(0).cpu().tolist()

    # اگر طول دنباله کمتر از max_len بود، با PAD پر می‌کنیم
    if len(seq) < max_len:
        seq = seq + [0] * (max_len - len(seq))  # 0 برای پر کردن PAD
    else:
        seq = seq[:max_len]

    return seq


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
file_path = "/content/drive/MyDrive/augmented_dataset.csv"
df = pd.read_csv(file_path)

df['clean_comment'] = df['comment'].apply(clean_text)

label_map = {-1: 0, 0: 1, 1: 2}
df['label'] = df['sentiment'].map(label_map)


In [5]:
df = df.sample(frac=1).reset_index(drop=True)
df

,comment,sentiment,clean_comment,label
0,مقایسه میزان فروش FIFA 14 و PES 14 (برای چند م...,-1,مقایسه میزان فروش FIFA 14 و PES 14 برای چند ما...,0
1,اصلابه صورت منظم موهای صورت را کوتاه نمی کنه و...,-1,اصلابه صورت منظم موهای صورت را کوتاه نمی کنه و...,0
2,سلام من سری قبلی این مارک رو خریدم از لحاظ سخ...,-1,سلام من سری قبلی این مارک رو خریدم از لحاظ سخ...,0
3,موارد ذکر شده زیر بعد از خرید و در روز اول است...,-1,موارد ذکر شده زیر بعد از خرید و در روز اول است...,0
4,آموزش بدونارتباط، کم انگیزه. می‌توان گفت مواد ...,0,آموزش بدونارتباط کم انگیزه میتوان گفت مواد عمل...,1
...,...,...,...,...
191835,این محصول تایمر نداره و متاسفانه در توضیحات اش...,-1,این محصول تایمر نداره و متاسفانه در توضیحات اش...,0
191836,دوستان مطمئن باشید بعد از یکماه مجبور میشید یه...,-1,دوستان مطمئن باشید بعد از یکماه مجبور میشید یه...,0
191837,دوره‌ای که همچنین انتقادات که جداگانه منظور کن...,0,دورهای که همچنین انتقادات که جداگانه منظور کنی...,1
191838,دوره‌ای که تمام توقعاتم را برآورده کرد، باورم ...,1,دورهای که تمام توقعاتم را برآورده کرد باورم آم...,2


In [6]:

class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = torch.tensor(self.texts[idx], dtype=torch.long)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return text, label



class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        self.attention = nn.Linear(input_dim, 1)

    def forward(self, x):
        scores = torch.tanh(self.attention(x))
        weights = torch.softmax(scores, dim=1)
        context = torch.sum(weights * x, dim=1)
        return context




class XLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout=0.5, pad_idx=0):
        super(XLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.spatial_dropout = nn.Dropout2d(0.2)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.attention_lstm = Attention(hidden_dim * 2)

        self.gru = nn.GRU(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.attention_gru = Attention(hidden_dim * 2)

        self.fc = nn.Linear(hidden_dim * 2 * 2, 64)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(64, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)

        embedded = embedded.permute(0, 2, 1)
        embedded = self.spatial_dropout(embedded)
        embedded = embedded.permute(0, 2, 1)

        lstm_out, _ = self.lstm(embedded)
        attn_lstm = self.attention_lstm(lstm_out)

        gru_out, _ = self.gru(embedded)
        attn_gru = self.attention_gru(gru_out)

        combined = torch.cat((attn_lstm, attn_gru), dim=1)
        x = self.fc(combined)
        x = torch.relu(x)
        x = self.dropout(x)
        output = self.out(x)
        return output

In [7]:
from tqdm import tqdm
max_words = 10000

all_tokens = []
for text in df['clean_comment'].values:
    tokens = tokenize_text(text)
    all_tokens.extend(tokens)

counter = Counter(all_tokens)
most_common = counter.most_common(max_words - 2)

word2idx = {"<PAD>": 0, "<UNK>": 1}
for idx, (word, _) in enumerate(most_common, start=2):
    word2idx[word] = idx

vocab_size = len(word2idx)
print("Dict size:", vocab_size)

max_len = 512

tqdm.pandas()

df['sequence'] = df['clean_comment'].progress_apply(lambda x: text_to_sequence(x, max_len))

X = np.array(df['sequence'].tolist())
y = np.array(df['label'].tolist())

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


Dict size: 10000


100%|██████████| 191840/191840 [1:02:12<00:00, 51.40it/s]


In [8]:
import pandas as pd

# مسیر ذخیره در گوگل درایو
file_path = "/content/drive/My Drive/embeded_dataset.csv"

# ذخیره DataFrame به CSV
df.to_csv(file_path, index=False)

print(f"File saved to {file_path}")

File saved to /content/drive/My Drive/embeded_dataset.csv


In [9]:
batch_size = 128

train_dataset = SentimentDataset(X_train, y_train)
val_dataset   = SentimentDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

In [19]:
embedding_dim = 512
hidden_dim = 128
output_dim = 3
pad_idx = word2idx["<PAD>"]

model = XLSTM(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim,
              output_dim=output_dim, dropout=0.5, pad_idx=pad_idx)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
print(len(df["sequence"][0]))

512


In [11]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_correct = 0
    total_train = 0

    for texts, labels in train_loader:
        texts = texts.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * texts.size(0)
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        total_train += texts.size(0)

    avg_train_loss = train_loss / total_train
    train_acc = train_correct / total_train


    model.eval()
    val_loss = 0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for texts, labels in val_loader:
            texts = texts.to(device)
            labels = labels.to(device)
            outputs = model(texts)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * texts.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            total_val += texts.size(0)

    avg_val_loss = val_loss / total_val
    val_acc = val_correct / total_val

    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} - "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1545: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
